In [23]:
"""imports"""
import os
from ansys.edb.database import Database
from ansys.edb.session import launch_session
import ansys.edb as edb
from ansys.edb.definition.djordjecvic_sarkar_model import DjordjecvicSarkarModel
import json
import psutil

In [24]:
"""configs"""
ROOT = os.getcwd()
EXE_ROOT = os.environ['ANSYSEM_ROOT232']
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT,'data','meshed_ground.aedb')
out = os.path.join(ROOT,'data','assign_ds_model.aedb')

In [25]:
"""launch session"""
PROCNAME = "EDB_RPC_Server.exe"
try:
    for proc in psutil.process_iter():
        if proc.name() == PROCNAME:
            proc.kill()
except:
    pass
session = launch_session(EXE_ROOT, 50051)

In [26]:
"""Open DB"""
db = Database.open(EDB_FILE,False)
cell = db.circuit_cells[0]
layout = cell.layout
layerStats = []
lc = layout.layer_collection
layers = lc.get_layers()
layer_names = [i.name for i in layers]

In [27]:
"""Assigning DS model"""
layer_info = []
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material().value
        thickness = i.thickness.value
        lower_elevation = i.lower_elevation.value
        full_material = i.get_fill_material().value
        
        if 'DIELECTRIC_LAYER' in str(layer_type):
            matmodel = edb.definition.material_def.MaterialDef.find_by_name(db,i.get_material().value)
            ds = DjordjecvicSarkarModel.create()
            matmodel.dielectric_material_model=ds            
        
db.save_as(out)

In [28]:
"""End of the session"""
session.disconnect()